#  Big Data Management Assignment 2

We answer the same questions which we had answered in Assignment 1 using Data frame API. But in this Assignment 2 we'll answer these questions using Apache Spark's SQL API. We had defined Cassandra structures and the Spark code that
saves the computed batch views into these structures.

Importing all sparkcontext, sparkconf,sparksession from pyspark 

In [312]:
from pyspark import SparkContext, SparkConf 
from pyspark.sql import SparkSession
sc = SparkContext.getOrCreate();
spark=SparkSession.builder.getOrCreate()
ss= spark

### 1) Loading the csv file from the Hdfs file system

Loading the csv file from hdfs requires the hadoop services to be started.

In [313]:
vehicle_records_DF = spark.read.csv("hdfs://localhost:9000/user/lalith/data/per-vehicle-records-2021-01-31.csv", inferSchema = True, header = True)


In [314]:
vehicle_records_DF.show()

+-----+----+-----+---+----+------+------+-----------+-----------+----+--------+------------+----------------+-----+---------+------+-------+----+-----+------+-----------+--------+------------+-------------+-----------+------------+
|cosit|year|month|day|hour|minute|second|millisecond|minuteofday|lane|lanename|straddlelane|straddlelanename|class|classname|length|headway| gap|speed|weight|temperature|duration|validitycode|numberofaxles|axleweights|axlespacings|
+-----+----+-----+---+----+------+------+-----------+-----------+----+--------+------------+----------------+-----+---------+------+-------+----+-----+------+-----------+--------+------------+-------------+-----------+------------+
|  998|2021|    1| 31|   2|    45|     0|          0|        165|   2|    Ch 2|           0|            null|    2|      CAR|   5.2|   1.07|1.13| 71.0|   0.0|        0.0|       0|           0|            0|       null|        null|
|  998|2021|    1| 31|   2|    45|     1|          0|        165|   2|  

Before performing the analysis on the Dataset, Intially we need to filter the dataset by identifying the M50 cosits from  junction 3 to junction 17 as per requirement.

Firstly, we need to identify the cosits on m50 highway by refering to the map of irish road traffic data and the dataset.
There are 15 counters present on m50 highway out of which 13 counters present in between junction 3 to 17. 

we need data for only those counters. so let's create a dataframe that contains only the required data by filtering the orginial dataset. 

In [315]:
cositlist = [1500,1501,1502,1508,1503,1509,1504,1505,1506,1507,15010,15011,15012]

m50cositdf = vehicle_records_DF.where(vehicle_records_DF.cosit.isin(cositlist))

In [316]:
type(m50cositdf)

pyspark.sql.dataframe.DataFrame

Creating a temporary table out from the data frame.

In [317]:
m50cositdf.registerTempTable("m50cositdf")


#### 1) Calculating the usage of Irish road network in terms of percentage by vehicle category

In [318]:
Irishroadusage = spark.sql("SELECT classname, COUNT(classname) AS count,\
round(count(classname)*100 / (select count(*) from m50cositdf),1) \
AS percentage from m50cositdf  GROUP BY classname ORDER BY percentage desc")


In [319]:
Irishroadusage.show()

+---------+------+----------+
|classname| count|percentage|
+---------+------+----------+
|      CAR|154927|      85.3|
|      LGV| 17860|       9.8|
|  HGV_ART|  5460|       3.0|
|  HGV_RIG|  1262|       0.7|
|      BUS|  1107|       0.6|
|  CARAVAN|   610|       0.3|
|    MBIKE|   355|       0.2|
|     null|     0|       0.0|
+---------+------+----------+



From the above query we can find the usage of irish road network by each vehicle class. From the result we can observe that classname CAR has the highest uasge with percentage of 85.321

In [320]:
Irishroadusage.write.format("org.apache.spark.sql.cassandra").options(table="roadusage2", keyspace="assignment")  


This line of code writes the result of the above spark sql code into the cassandra table called roadusage2 we had defined earlier. 

#### 2) Calculating the highest and lowest hourly fows on M50 - show the hours and total number of vehicle counts.

In [321]:
hourlyflow = spark.sql("SELECT hour, count(hour) AS count from m50cositdf GROUP BY hour ORDER BY count desc ")

In [322]:
hourlyflow.show(24)

+----+-----+
|hour|count|
+----+-----+
|  15|17211|
|  16|16575|
|  14|16373|
|  13|15784|
|  17|14742|
|  12|11768|
|  18|11503|
|  11|10943|
|  19| 9769|
|  20| 9437|
|  10| 8310|
|  21| 6426|
|   9| 6055|
|   7| 5720|
|   8| 4922|
|   6| 3483|
|  22| 3389|
|  23| 2664|
|   0| 1844|
|   5| 1288|
|   1| 1187|
|   2|  891|
|   4|  788|
|   3|  510|
+----+-----+



The above query gives the count of vehicles per hour, we can observe that hours 13 -17 has highest uage, while hours 1 - 5 has least uasge

In [323]:
hourlyflow.write.format("org.apache.spark.sql.cassandra").options(table="flow", keyspace="assignment").save(mode="append") 

This line of code writes the result of the above spark sql code into the cassandra table called flow, which we had defined earlier. 

#### 3) Calculating the evening and morning rush hours on M50 - show the hours and the total counts

In [324]:
morning_hours = [9,10,11]
evening_hours = [20,21,22]

In [325]:
morninghours = spark.sql("SELECT hour, count(hour) AS count from m50cositdf WHERE hour IN (9,10,11)\
GROUP BY hour ORDER BY count desc ")

In [326]:
morninghours.show()

+----+-----+
|hour|count|
+----+-----+
|  11|10943|
|  10| 8310|
|   9| 6055|
+----+-----+



The above query identifies the morning rush hours and the count of vehicles per that hour. We can observe that morning rush hours are between 9 -12

In [327]:
morninghours.write.format("org.apache.spark.sql.cassandra").\
options(table="mrushhours", keyspace="assignment").save(mode="append") 

This line of code writes the result of the above spark sql code into the cassandra table called "mrushhours", which we had defined earlier. 

In [328]:
eveninghours = spark.sql("SELECT hour, count(hour) AS count from m50cositdf WHERE hour IN (20,21,22)\
GROUP BY hour ORDER BY count desc ")

In [329]:
eveninghours.show()

+----+-----+
|hour|count|
+----+-----+
|  20| 9437|
|  21| 6426|
|  22| 3389|
+----+-----+



The above query identifies the Evening rush hours and the count of vehicles per that hour. We can observe that evening rush hours are between 20 -22

In [330]:
eveninghours.write.format("org.apache.spark.sql.cassandra").\
options(table="erushhours", keyspace="assignment").save(mode="append") 

This line of code writes the result of the above spark sql code into the cassandra table called "erushhours", which we had defined earlier. 

#### 4) Calculating the average speed between each junctions on M50


In [331]:
averagespeed = spark.sql("SELECT cosit, round(AVG(speed),2) AS avgspeed from m50cositdf GROUP BY cosit")

In [332]:
averagespeed.show()

+-----+--------+
|cosit|avgspeed|
+-----+--------+
| 1507|  102.74|
| 1500|   93.75|
| 1506|  102.79|
| 1505|   99.69|
| 1504|  101.99|
|15010|  105.02|
|15012|   105.1|
| 1509|   98.35|
| 1502|  102.36|
|15011|   101.8|
| 1501|  101.33|
| 1503|  102.18|
| 1508|   98.65|
+-----+--------+



The above the query gives the average vehicle speed recorded at each counter.

In [333]:
averagespeed.write.format("org.apache.spark.sql.cassandra").\
options(table="avg_speed", keyspace="assignment").save(mode="append") 

This line of code writes the result of the above spark sql code into the cassandra table called "avg_speed", which we had defined earlier. 

Here i have also added junction names of the each cosit to the result 

In [334]:
junctionlist =[("Junction 3-4", 1500),("Junction 4-5", 1501),("Junction 5-6", 1502),("Junction 6-7",1508),("Junction 7-9",1503),("Junction 9-10",1509),("Junction 10-11",1504),("Junction 11-12",1505),("Junction 12-13",1506),("Junction 13-14",1507),("Junction 14-15",15010),("Junction 15-16",15011),("Junction 16-17",15012)]
jun= sc.parallelize(junctionlist).collect()
juncdataf=spark.createDataFrame(jun, ["junction","cosit"])
juncdataf.show()

+--------------+-----+
|      junction|cosit|
+--------------+-----+
|  Junction 3-4| 1500|
|  Junction 4-5| 1501|
|  Junction 5-6| 1502|
|  Junction 6-7| 1508|
|  Junction 7-9| 1503|
| Junction 9-10| 1509|
|Junction 10-11| 1504|
|Junction 11-12| 1505|
|Junction 12-13| 1506|
|Junction 13-14| 1507|
|Junction 14-15|15010|
|Junction 15-16|15011|
|Junction 16-17|15012|
+--------------+-----+



In [335]:
juncdataf.registerTempTable("juncdataf")

In [336]:
avgspeedbetweenjunc= spark.sql("SELECT m50cositdf.cosit,round(AVG(m50cositdf.speed),1) AS avgspeed,\
juncdataf.junction from m50cositdf JOIN juncdataf ON m50cositdf.cosit = juncdataf.cosit \
GROUP BY m50cositdf.cosit,juncdataf.junction ");

In [ ]:
avgspeedbetweenjunc.show()

This is a refined output for this question, I have added the  junction numbers at which the respective counter is located and the percentage is rounded to 1 decimal.

In [ ]:
avgspeedbetweenjunc.write.format("org.apache.spark.sql.cassandra").\
options(table="avgspeedatjunction", keyspace="assignment").save(mode="append") 

This line of code writes the result of the above spark sql code into the cassandra table called "avg_speed", which we had defined earlier. 

#### 5) Calculating the top 10 locations with highest number of counts of HGVs(class)and Mapping  the COSITs with their names given on the map

In [ ]:
locationlist =[("TMU M50 001.7 N", 1500),("TMU M50 005.0 N", 1501),("TMU M50 010.0 N", 1502),("TMU M50 015.0 S",1508),("TMU M50 020.0 N",1503),("TMU M50 015.0 N",1509),("TMU M50 025.0 S",1504),("TMU M50 025.0 N",1505),("TMU M50 030.0 S",1506),("TMU M50 035.0 S",1507),("TMU M50 040.0 S",15010),("TMU M50 035.0 N",15011),("TMU M50 040.0 N",15012)]
nm= sc.parallelize(locationlist).collect()
dataf=spark.createDataFrame(nm, ["location","cosit"])
dataf.show()

In [ ]:
dataf.registerTempTable("dataf")

We have created a new samll dataframe called dataf, It contains the names of the location for each cosit. This dataframe has been converted to temporary table for running the sql queries.

In [ ]:
toplocation = spark.sql("SELECT  Distinct m50cositdf.cosit,  count(m50cositdf.cosit) AS count,\
dataf.location from m50cositdf JOIN dataf ON m50cositdf.cosit = dataf.cosit \
WHERE m50cositdf.classname ='HGV_RIG' OR m50cositdf.classname = 'HGV_ART'  \
GROUP BY m50cositdf.cosit, dataf.location ORDER BY count desc");


In [ ]:
toplocation.show(10)

We have joined the two temporary tables to the get the location name next to the cosit, The above query gives the top 10 locations and its cosits with highest counts of vehicles class "HGV".

In [ ]:
toplocation.write.format("org.apache.spark.sql.cassandra").\
options(table="toplocations", keyspace="assignment").save(mode="append") 

This line of code writes the result of the above spark sql code into the cassandra table called "toplocations", which we had defined earlier. 